### 1. Importing Required Modules and Packages

In [2]:
import os
import sys

sys.path.append('..')
os.environ["OMP_NUM_THREADS"] = '1'  # KMeans is not parallelized, so set to 1 thread

from src.mutation import Mutation
from src.sequence import Plasmid
from src.eblocks import Eblock, EblockDesign
import biotite.sequence as seq
from src.primer import DesignPrimers
from src.plot import Plot
from src.utils import Utils, SnapGene

%reload_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def circular_mean(values):
    """ Calculate the circular mean of a list of values. """
    radians = np.deg2rad(values)  # Convert degrees to radians
    sin_mean = np.mean(np.sin(radians))
    cos_mean = np.mean(np.cos(radians))
    mean_angle = np.arctan2(sin_mean, cos_mean)  # Get the mean angle in radians
    return np.rad2deg(mean_angle) % 360  # Convert back to degrees

# Step 1: Define Your Data
data = np.array([10, 20, 30, 40, 7000, 7010, 7020, 7030])  # Unique numbers
pairs = [(10, 7000), (20, 30)]  # Pairs that should be in the same cluster

# Step 2: Transform the Data
# Create new dataset based on pairs and include unpaired numbers
paired_data = []
for pair in pairs:
    paired_data.append(circular_mean(pair))  # Use circular mean of each pair for clustering

# Include unpaired numbers in the dataset
for number in data:
    if all(number not in pair for pair in pairs):  # Check if number is not part of any pair
        paired_data.append(number)

paired_data = np.array(paired_data).reshape(-1, 1)  # Reshape for clustering

# Step 3: Apply KMeans Clustering
n_clusters = 3  # Adjust based on your specific data and requirements
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(paired_data)

# Get cluster labels for each transformed point
cluster_labels = kmeans.labels_

# Step 4: Map Results Back to Original Data
# Create a complete cluster label mapping
original_data = np.array([10, 20, 30, 40, 7000, 7010, 7020, 7030])
final_labels = {}

# Assign cluster labels for pairs
for pair in pairs:
    cluster_label = cluster_labels[pairs.index(pair)]  # Get cluster label for the pair's mean
    final_labels[pair[0]] = cluster_label
    final_labels[pair[1]] = cluster_label

# Assign cluster labels for individual numbers
for number in original_data:
    if number not in final_labels:
        # Find the index of the number in paired_data
        if number in paired_data.flatten():
            index = np.where(paired_data.flatten() == number)[0][0]
            final_labels[number] = cluster_labels[index]
        else:
            # If it's not a paired number, it should just get the last cluster label
            transformed_index = len(pairs) + np.where(np.isin(paired_data.flatten(), original_data[:len(paired_data.flatten())]))[0].size
            final_labels[number] = cluster_labels[transformed_index]

# Display the final mapping of original numbers to cluster labels
print("Final Cluster Labels for Original Numbers:", {k: final_labels[k] for k in sorted(final_labels)})

# Step 5: Visualization
# Prepare the data for visualization
visual_labels = [final_labels[number] for number in original_data]

# Plotting the results
plt.scatter(original_data, np.zeros_like(original_data), c=visual_labels, cmap='viridis', s=100)
plt.title('Cluster Visualization of Mixed Pairs and Individual Numbers with Circularity')
plt.xlabel('Original Data Points')
plt.yticks([])  # Hide y-ticks
plt.show()

c:\Users\Rosan\Anaconda3\envs\eBlocks\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Final Cluster Labels for Original Numbers: {10: np.int32(2), 20: np.int32(0), 30: np.int32(0), np.int64(40): np.int32(0), 7000: np.int32(2), np.int64(7010): np.int32(1), np.int64(7020): np.int32(1), np.int64(7030): np.int32(1)}


ValueError: object __array__ method not producing an array

<Figure size 640x480 with 1 Axes>

### 2. Loading and Analyzing the Gene Sequence

The desired mutations should be added to a txt file. 
Here, we create a Mutation() object and parse the specified mutations. 

In [ ]:
# Create a Mutation object and parse the input mutations from the files/ directory

mutations_file = 'files/mutations.txt'

mutation_instance = Mutation()
mutation_instance.parse_mutations(mutations_file)

# Print the mutations that were parsed
mutation_instance.print_mutations()

Next, we read the gene sequence and the vector that contains our gene of interest.

In [ ]:
# Create a Plasmid object and parse the input plasmid from the files/ directory

sequence_file = 'files/A0QX55.fasta'
vector_file = 'files/vector.dna'

sequence_instance = Plasmid()
sequence_instance.parse_vector(vector_file)
sequence_instance.parse_sequence(sequence_file)

We also define an output directory for the generated files and create a snapgene object for visualization

In [5]:
# Create a SnapGene instance to write the eBlocks features to a snapgene file

# Set output directory
output_dir = '0925-output'

snapgene_instance = SnapGene(sequence_instance=sequence_instance,
                             output_dir=output_dir)

We create an eBlockDesign instance that can initiate the design of the eblocks. Here, we choose as optimization method cost_optimization that aims to use as little basepairs as possible. Another option would be to do amount_optimization, that aims to cluster as many mutations as possible together, to get the lowest number of different eBlocks

In [6]:
# Create an Eblocks object based on the input mutations and the gene sequence

design_instance = EblockDesign(mutation_instance=mutation_instance,
                               sequence_instance=sequence_instance,
                               output_dir=output_dir,
                               verbose=True,
                               cost_optimization=False,
                               amount_optimization=True)

In [ ]:
# Create a Plots object and check the input vector

# Create the Plot object
plot_instance = Plot(mutation_instance=mutation_instance,
                     eblocks_design_instance=design_instance,
                     sequence_instance=sequence_instance,
                     output_dir=output_dir,
                     show=True)

# Check the input vector
plot_instance.plot_vector(figsize=(5, 5));

In [8]:
# TODO Add DnaE1 gene sequence to vector
# TODO What are the other things in the vector that do not have a name?

In our vector we can see that our vector contains the SacB gene, has an origin of replication and contains a CmR (chloramphenicol) resistance marker

In [9]:

# TODO Show eBlocks in vector as well
# TODO Add plasmid visaulization of eBlock features


# from Bio import SeqIO
# from Bio.Graphics import GenomeDiagram
# from Bio.SeqFeature import SeqFeature, FeatureLocation

# # Parse the plasmid sequence
# plasmid_seq_record = SeqIO.read("plasmid_sequence.fasta", "fasta")

# # Create a GenomeDiagram object
# gd_diagram = GenomeDiagram.Diagram("Plasmid Map")

# # Add the sequence track
# gd_track = gd_diagram.new_track(1, name="Plasmid")
# gd_feature_set = gd_track.new_set()

# # Add the plasmid sequence
# gd_feature_set.add_feature(SeqFeature(FeatureLocation(0, len(plasmid_seq_record))), color="black")

# # Parse the GFF3 file to extract features
# # Assuming you have a function parse_gff3() that returns feature information
# features = parse_gff3("plasmid_features.gff3")

# # Add the features to the plasmid map
# for feature in features:
#     start = feature.start
#     end = feature.end
#     name = feature.attributes["Name"]
#     gd_feature_set.add_feature(SeqFeature(FeatureLocation(start, end)), color="blue", label=True, label_position="middle", label_size=8, label_angle=0, label_strand=0, name=name)

# # Draw the plasmid map
# gd_diagram.draw(format="linear", pagesize=(15*len(plasmid_seq_record), 400), fragments=1)
# gd_diagram.write("plasmid_map.png", "png")


In [ ]:
# Run the eBlocks design and print the results

design_instance.run_design_eblocks()

In the process, for each mutation a different eBlock is created and a .gb file is made to easily view the clone in a sequence editor. 

In [ ]:
# Now that we have designed the eblocks, we can visualize them using the Plot class

plot_instance.plot_eblocks_mutations(figure_length=20,
                                     figure_width=5)

In [ ]:
sequence_file = 'files/A0QX55.fasta'
vector_file = 'files/vector.dna'

sequence_instance = Plasmid()
sequence_instance.parse_vector(vector_file)
sequence_instance.parse_sequence(sequence_file)

sequence_instance.description

In [23]:
from Bio import SeqIO

def read_single_fasta(fp: str) -> str:
    """
    This function reads a single fasta file and returns the sequence.
    """
    for num, record in enumerate(SeqIO.parse(fp, "fasta")):
        sequence = record.seq
        seqid = record.id
        if num > 0:
            raise ValueError("Please provide a single sequence in FASTA format.")
    return sequence, seqid

def read_single_fasta(fp: str) -> str:
    """
    This function reads a single fasta file and returns the sequence.
    """
    record = next(SeqIO.parse(fp, "fasta"))  # Read the first and only record
    sequence = record.seq
    seqid = record.id
    return sequence, seqid

In [ ]:
seq, seqid = read_single_fasta(sequence_file)
print(seq, seqid)
seq, seqid = read_single_fasta2(sequence_file)
print(seq, seqid)